# Preparation

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from ipywidgets import FloatProgress
from IPython.display import display

In [ ]:
pd.__version__

In [ ]:
np.__version__

### common variables

In [ ]:
checkpointsStem = "tutorial1.Checkpoint";

### columns

In [ ]:
# In RedMetrics data, 'playerId' is actually a session ID.
# Permanent player IDs are stored as 'localplayerguid' in 'customdata' attached to 'start' events.
minimalInitializationColumns = ['customData.localplayerguid']
initializationColumns = ['customData.localplayerguid', 'playerId', 'type', 'serverTime', 'customData.platform']
relevantColumns = ['sessionId', 'serverTime', 'userId', 'customData.platform']
playerFilteringColumns = ['sessionId', 'userId', 'customData.platform', 'serverTime']
checkpointsRelevantColumns = ['sessionId', 'userId', 'type', 'section']

### dataframes for the different game versions

In [ ]:
def getNormalizedRedMetricsCSV( df ):
    newColumns = np.unique(np.concatenate((minimalInitializationColumns, df.columns.values)))
    return safeGetNormalizedRedMetricsCSV(df.loc[:,newColumns])

def safeGetNormalizedRedMetricsCSV( df ):
    return df.rename(index=str, columns={'customData.localplayerguid' : 'userId', 'playerId': 'sessionId'})

In [ ]:
relevantColumns = ['customData.localplayerguid', 'playerId', 'type']
renamedRelevantColumns = ['userId', 'sessionId', 'type']

# Old versions

rdf100 = pd.read_csv("../../data/1.0.csv")
rdf131 = pd.read_csv("../../data/1.31.csv")
rdf132 = pd.read_csv("../../data/1.32.csv")
rdf133 = pd.read_csv("../../data/1.33.csv")
rdf140 = pd.read_csv("../../data/1.40.csv")
rdf150 = pd.read_csv("../../data/1.50.csv")
rdf151 = pd.read_csv("../../data/1.51.csv")

part100 = rdf100.loc[:,relevantColumns]
part131 = rdf131.loc[:,relevantColumns]
part132 = rdf132.loc[:,relevantColumns]
part133 = rdf133.loc[:,relevantColumns]
part140 = rdf140.loc[:,relevantColumns]
part150 = rdf150.loc[:,relevantColumns]
part151 = rdf151.loc[:,relevantColumns]

# Tests

rdftest = pd.read_csv("../../data/2017-10-11.test.csv")
dftest = getNormalizedRedMetricsCSV(rdftest)

# Version 1.52

In [ ]:
dataFilesNamesStem = "2018-01-31"

rdf152  = pd.read_csv("../../data/" + dataFilesNamesStem + ".1.52.csv")
rdftest = pd.read_csv("../../data/" + dataFilesNamesStem + ".test.csv")

part152 = rdf152.loc[:,relevantColumns]

normalizedDF152  = getNormalizedRedMetricsCSV(rdf152)
normalizedDFTest = getNormalizedRedMetricsCSV(rdftest)

TOD: get rid of warning
    
    DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
    interactivity=interactivity, compiler=compiler, result=result)

using https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

## Filtering

In [ ]:
def getAllSessions( _rmDF, dropna ):
    _result = _rmDF.loc[:, renamedRelevantColumns]
    _result = _result[_result['type']=='start']
    _result = _result.drop('type', 1)
    if dropna:
        _result = _result.dropna(how='any')
    return _result

In [ ]:
# gets sessions which either:
#  - have 'android' or '...editor' as platform
#  - are in the RedMetrics test channel
#  - belong to a user who has a session of the type above
def getTestSessions(_rmDF, _rmTestDF):

    # tables of association of user-sessions
    rmDFUsersSessions = getAllSessions(_rmDF, False)
    rmTestDFUsersSessions = getAllSessions(_rmTestDF, False)
    userSessions = pd.concat([rmDFUsersSessions,rmTestDFUsersSessions])

    #  - have 'android' or '...editor' as platform
    rmDFTestSessions = set(_rmDF[_rmDF['customData.platform'].apply(lambda s: str(s).endswith('editor"'))]['sessionId'])
    rmDFTestSessions |= set(_rmDF[_rmDF['customData.platform'].isin(['"android"'])]['sessionId'])
    #print(str(len(rmDFTestSessions)))

    #  - are in the RedMetrics test channel
    rmTestDFTestSessions = set(_rmTestDF['sessionId'])
    #print(str(len(rmTestDFTestSessions)))

    #  - belong to a user who has a session of the type above
    # all the sessions above
    testSessions = rmDFTestSessions | rmTestDFTestSessions
    # all the users
    rmDFTestUsers = set(rmDFUsersSessions[rmDFUsersSessions['sessionId'].isin(testSessions)]['userId'].dropna())
    rmTestDFTestUsers = set(_rmTestDF['userId'].dropna())
    testUsers = rmDFTestUsers | rmTestDFTestUsers
    # all the sessions which belong to these users
    result = set(rmDFUsersSessions[rmDFUsersSessions['userId'].isin(testUsers)]['sessionId'].dropna())

    return result

In [ ]:
testSessions = getTestSessions(_rmDF = normalizedDF152, _rmTestDF = normalizedDFTest)
df152 = normalizedDF152[~normalizedDF152['sessionId'].isin(testSessions)]

len(normalizedDF152['sessionId'].unique()),len(testSessions),len(df152['sessionId'].unique())

## Data format fixes
### date string to pd.Timestamp

In [ ]:
df152.loc[:,'userTime'] = df152.loc[:,'userTime'].apply(pd.to_datetime)
df152.loc[:,'serverTime'] = df152.loc[:,'serverTime'].apply(pd.to_datetime)

### user id string simplification
User Id follows the pattern *'"id"'* because it is a *customData*

In [ ]:
#userSessions = df152.loc[:,['userId','sessionId']]
#for userSessionsIndex in userSessions.index:
#    print(userSessions.loc[userSessionsIndex])

In [ ]:
#df152.loc[:,'userId'] = df152.loc[:,'userId'].apply(lambda s: str(s).replace('"',''))

# All versions

rdf = pd.concat([part100, 
                      part131, part132, part133, 
                      part140, 
                      part150, part151, part152])

df = getNormalizedRedMetricsCSV(rdf)